In [ ]:
from google.genai import types
from google.adk.agents.llm_agent import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor

print("✅ ADK components imported successfully.")

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from datetime import datetime, timedelta
from typing import Optional

cred = credentials.Certificate("lanaagent-firebase-adminsdk-fbsvc-3ed45f99b9.json")
firebase_admin.initialize_app(cred)

db=firestore.client()

from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

✅ ADK components imported successfully.


ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [17]:
#helpers and tools

retry_config = types.HttpRetryOptions(
    attempts=3,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

def log_persistence(log_entry_data: dict, is_authenticated: bool) -> bool:
    """
    Saves a structured log entry to the Google Cloud Persistence Store (Firestore).
    Includes a Contact Gate check and an auto-generated timestamp.

    It also controls the sharing of contact information based on authentication status.

    Args:
        log_entry_data (dict): The log entry dictionary with this structure:
            {
                "timestamp": "ISO 8601",
                "category": "Structural/Time, Persona, Avoidance, or Project",
                "key": "The specific field name (e.g., 'Preferred Communication Tone')",
                "value": "The current data value",
                "source": "e.g., 'User Input', 'Bujo OCR', 'Agent Check'"
            }
        is_authenticated (bool): Boolean flag indicating if the user is logged into 
                          their Google Account (True) or anonymous (False).

    Returns:
        bool: True if the log was successfully written; False otherwise.
    """

    
    # 1. Structural Check: The Contact Gate (for ethical protection)
    if log_entry_data.get('key') == 'contact_info' and not is_authenticated:
        print("ACCESS DENIED: Cannot share contact info for anonymous user.")
        # We do not log the entry to prevent accidental data leakage
        return False
    
    try:
        # 2. Add Required Metadata (Timestamp)
        # Create a new dictionary to ensure we always log a timestamp
        data_to_log = log_entry_data.copy()
        data_to_log['timestamp'] = datetime.utcnow()
        
        # 3. Cloud Write: Writing the data to the 'logs' collection in Firestore
        db.collection('logs').add(data_to_log)
        
        print(f"SUCCESS: Logged data to Firestore: {log_entry_data.get('key')}")
        return True
        
    except Exception as e:
        print(f"FAILURE: Could not write to Firestore. Error: {e}")
        return False

In [ ]:
#tools part 2
from datetime import datetime, timedelta
from typing import Optional
def query_persistence(task_id: str) -> dict:
    """
    Queries the Persistence Store (Firestore) for a specific task's history 
    and checks if the Contradiction Flag should be raised.
    
    Args:
        task_id: The unique identifier (Task Key) of the task to search for.

    Returns:
        A dictionary containing the check result and failure count.
    """
    
    logs_ref = db.collection('logs').where('key', '==', task_id)
    
    flag_raised = True if logs_ref is None else False
    
    # Simulate a successful query that found 4 failures in a row
    consecutive_failure_count = 4 

    # THE CONTRADICTION FLAG LOGIC 
    flag_raised = consecutive_failure_count >= 3

    return {
        "task_id": task_id,
        "consecutive_failures": consecutive_failure_count,
        "flag_raised": flag_raised
    }

def _parse_date_token(date_str: str) -> datetime:
    """
    Parse a date string that may be either an ISO date or a relative token.

    Supported formats:
    - ISO date: 'YYYY-MM-DD' (e.g., '2025-11-24')
    - Relative tokens: 'NOW', 'TODAY', 'NOW-7DAYS', 'NOW-30DAYS' (case-insensitive)

    Returns a timezone-naive UTC datetime at the start of the day.
    Raises ValueError for unsupported formats.
    """
    if not isinstance(date_str, str):
        raise ValueError("date_str must be a string")

    token = date_str.strip().upper()
    if token in ("NOW", "TODAY"):
        return datetime.utcnow().replace(hour=0, minute=0, second=0, microsecond=0)

    # Handle patterns like NOW-7DAYS or TODAY-1DAYS
    if (token.startswith("NOW-") or token.startswith("TODAY-")) and "DAYS" in token:
        try:
            # last part expected to be like '7DAYS' -> extract leading integer
            n_part = token.split("-")[-1]
            n = int(''.join(ch for ch in n_part if ch.isdigit()))
            return (datetime.utcnow() - timedelta(days=n)).replace(hour=0, minute=0, second=0, microsecond=0)
        except Exception as e:
            raise ValueError(f"Could not parse relative date token: {date_str}") from e

    # Fallback: try ISO format
    try:
        return datetime.strptime(token, "%Y-%m-%d")
    except Exception as e:
        raise ValueError(f"Unsupported date format: {date_str}. Use 'YYYY-MM-DD' or 'NOW-<NDAYS>'") from e


def make_a_report(start_date_str: str, end_date_str: str) -> dict:
    """
    Queries the Persistence Store (Firestore) for a date span and returns matching logs.

    Accepted date formats:
    - ISO date: 'YYYY-MM-DD' (e.g., '2025-11-01')
    - Relative tokens: 'NOW', 'TODAY', 'NOW-7DAYS' (case-insensitive)

    Args:
        start_date_str: Start date string or relative token.
        end_date_str: End date string or relative token.

    Returns:
        dict: { 'logs': [ ... ] }
    """
    # Parse inputs (accept tokens like NOW-7DAYS)
    start_date = _parse_date_token(start_date_str)
    end_date = _parse_date_token(end_date_str)

    # Ensure end_date includes the full day
    end_date = end_date.replace(hour=23, minute=59, second=59, microsecond=999999)

    # Execute the query and stream the data
    logs_stream = db.collection('logs').where('timestamp', '>', start_date).where('timestamp', '<', end_date).stream()

    # Return the data as a list of dictionaries
    logs_data = [log.to_dict() for log in logs_stream]

    return {
        "logs": logs_data
    }

def write_project(project_data: dict) -> dict:
    """
    Write or update a project document to the `projects` collection in Firestore.
    """
    try:
        if not isinstance(project_data, dict):
            raise ValueError("project_data must be a dict")

        # Required fields
        if not project_data.get("project_name") or not project_data.get("project_type"):
            raise ValueError("project_data must include 'project_name' and 'project_type'")

        data = project_data.copy()

        # Normalize date fields if provided as strings
        for date_field in ("deadline", "created_date", "completed_date"):
            if date_field in data and isinstance(data[date_field], str):
                try:
                    data[date_field] = datetime.strptime(data[date_field], "%Y-%m-%d")
                except Exception:
                    pass

        # Defaults
        data.setdefault("status", "active")
        data.setdefault("actual_hours", 0)
        data.setdefault("created_date", datetime.utcnow())

        # If doc_id provided, update existing doc
        doc_id = data.pop("doc_id", None)
        if doc_id:
            doc_ref = db.collection("projects").document(str(doc_id))
            doc_ref.set(data, merge=True)
            return {"success": True, "doc_id": doc_id, "data": data}
        else:
            doc_ref = db.collection("projects").add(data)[1]
            try:
                new_doc_id = doc_ref.id
            except Exception:
                new_doc_id = None
            return {"success": True, "doc_id": new_doc_id, "data": data}

    except Exception as e:
        return {"success": False, "error": str(e)}


def read_projects(filter_by: Optional[dict] = None, limit: int = 100) -> dict:
    """
    Read projects from the `projects` collection.

    Args:
        filter_by (dict): optional mapping of field -> value to filter with equality.
            e.g. { 'status': 'active', 'project_type': 'tech' }
        limit (int): maximum number of documents to return.

    Returns:
        dict: { 'success': bool, 'projects': [dict,...], 'error': str (if any) }
    """
    try:
        coll = db.collection("projects")
        query = coll
        if filter_by:
            if not isinstance(filter_by, dict):
                raise ValueError("filter_by must be a dict mapping field->value")
            for k, v in filter_by.items():
                query = query.where(k, "==", v)

        docs = query.limit(limit).stream()
        projects = []
        for d in docs:
            doc = d.to_dict()
            doc["_doc_id"] = d.id
            projects.append(doc)

        return {"success": True, "projects": projects}
    except Exception as e:
        return {"success": False, "error": str(e)}


def write_category(category_data: dict) -> dict:
    """
    Write or update a category document to the `categories` collection in Firestore.
    """
    try:
        if not isinstance(category_data, dict):
            raise ValueError("category_data must be a dict")

        if not category_data.get("category_name"):
            raise ValueError("category_data must include 'category_name'")

        data = category_data.copy()

        if "is_active" in data:
            try:
                data["is_active"] = bool(int(data["is_active"])) if isinstance(data["is_active"], (str, int)) else bool(data["is_active"])
            except Exception:
                data["is_active"] = True
        else:
            data.setdefault("is_active", True)

        doc_id = data.pop("doc_id", None)
        if doc_id:
            doc_ref = db.collection("categories").document(str(doc_id))
            doc_ref.set(data, merge=True)
            return {"success": True, "doc_id": doc_id, "data": data}
        else:
            add_result = db.collection("categories").add(data)
            new_doc_id = None
            try:
                new_doc_ref = add_result[0]
                new_doc_id = new_doc_ref.id
            except Exception:
                try:
                    new_doc_id = add_result.id
                except Exception:
                    new_doc_id = None
            return {"success": True, "doc_id": new_doc_id, "data": data}

    except Exception as e:
        return {"success": False, "error": str(e)}


def read_categories(filter_by: Optional[dict] = None, limit: int = 100) -> dict:
    """
    Read categories from the `categories` collection.

    Args:
        filter_by (dict): optional mapping of field -> value to filter with equality.
            e.g. { 'is_active': True }
        limit (int): maximum number of documents to return.

    Returns:
        dict: { 'success': bool, 'categories': [dict,...], 'error': str (if any) }
    """
    try:
        coll = db.collection("categories")
        query = coll
        if filter_by:
            if not isinstance(filter_by, dict):
                raise ValueError("filter_by must be a dict mapping field->value")
            for k, v in filter_by.items():
                query = query.where(k, "==", v)

        docs = query.limit(limit).stream()
        categories = []
        for d in docs:
            doc = d.to_dict()
            doc["_doc_id"] = d.id
            categories.append(doc)

        return {"success": True, "categories": categories}
    except Exception as e:
        return {"success": False, "error": str(e)}


def write_task(task_data: dict) -> dict:
    """
    Write or update a task (milestone) document to the `project_tasks` collection in Firestore.
    """
    try:
        if not isinstance(task_data, dict):
            raise ValueError("task_data must be a dict")

        if not task_data.get("project_id") or not task_data.get("milestone_name"):
            raise ValueError("task_data must include 'project_id' and 'milestone_name'")

        data = task_data.copy()

        for date_field in ("due_date", "completed_date"):
            if date_field in data and isinstance(data[date_field], str):
                try:
                    data[date_field] = datetime.strptime(data[date_field], "%Y-%m-%d")
                except Exception:
                    pass

        data.setdefault("status", "pending")

        doc_id = data.pop("doc_id", None)
        if doc_id:
            doc_ref = db.collection("project_tasks").document(str(doc_id))
            doc_ref.set(data, merge=True)
            return {"success": True, "doc_id": doc_id, "data": data}
        else:
            add_result = db.collection("project_tasks").add(data)
            new_doc_id = None
            try:
                new_doc_ref = add_result[0]
                new_doc_id = new_doc_ref.id
            except Exception:
                try:
                    new_doc_id = add_result.id
                except Exception:
                    new_doc_id = None

            return {"success": True, "doc_id": new_doc_id, "data": data}

    except Exception as e:
        return {"success": False, "error": str(e)}


def read_tasks(filter_by: Optional[dict] = None, limit: int = 100) -> dict:
    """
    Read tasks (milestones) from the `project_tasks` collection.
    """
    try:
        coll = db.collection("project_tasks")
        query = coll
        if filter_by:
            if not isinstance(filter_by, dict):
                raise ValueError("filter_by must be a dict mapping field->value")
            for k, v in filter_by.items():
                query = query.where(k, "==", v)

        docs = query.limit(limit).stream()
        tasks = []
        for d in docs:
            doc = d.to_dict()
            doc["_doc_id"] = d.id
            tasks.append(doc)

        return {"success": True, "tasks": tasks}
    except Exception as e:
        return {"success": False, "error": str(e)}

✅ log_analyst_agent created with custom function tools
🔧 Available tools:


In [ ]:
root_agent = Agent(
    name='root_agent',
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    description='Lanas personal assistant',
    instruction="""You are a productivity log analyst and objective structural evaluator. 
    Your sole purpose is to analyze the Persistence Store data using your tools and provide objective assessments.
    
    If asked about commitment status, run the 'query_persistence' tool to check the Contradiction Flag.
    If asked for a summary (daily, weekly, monthly), run the 'make_a_report' tool and provide structural insights based on logged avoidance patterns and corrective actions.
    Your output must be non-judgmental and focused on systemic improvement.
    """,
    tools=[
        log_persistence,
        AgentTool(agent=log_analyst_agent),
        query_persistence,
        make_a_report,
        write_project,
        read_projects,
        write_task,
        read_tasks,
        write_category,
        read_categories,
    ]
)
print("✅ Root Agent defined.")

✅ Root Agent defined.


In [24]:
runner = InMemoryRunner(agent=root_agent)

print("✅ Runner created.")

✅ Runner created.


In [9]:
response = await runner.run_debug("m tired and missed most focus tasks in afternoon feel too tired to study.")


 ### Continue session: debug_session_id

User > m tired and missed most focus tasks in afternoon feel too tired to study.


C:\Users\Owner\AppData\Local\Temp\ipykernel_19480\3276699925.py:44: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  data_to_log['timestamp'] = datetime.utcnow()


SUCCESS: Logged data to Firestore: Energy Levels
root_agent > The log entry regarding your afternoon energy levels and missed focus tasks has been recorded.


In [ ]:

# Test the log_analyst_agent
log_analyst_runner = InMemoryRunner(agent=log_analyst_agent)
_ = await log_analyst_runner.run_debug(
    "what was completed today from 2025-11-23 to 2025-11-24"
)



In [26]:
response = await runner.run_debug("can you provide a summary of my productivity logs for the past week?")


 ### Continue session: debug_session_id

User > can you provide a summary of my productivity logs for the past week?


C:\Users\Owner\AppData\Local\Temp\ipykernel_19480\3067122689.py:47: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(hour=0, minute=0, second=0, microsecond=0)
C:\Users\Owner\AppData\Local\Temp\ipykernel_19480\3067122689.py:55: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return (datetime.utcnow() - timedelta(days=n)).replace(hour=0, minute=0, second=0, microsecond=0)
C:\Users\Owner\AppData\Local\Temp\ipykernel_19480\3067122689.py:89: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  logs_stream = db.collection('logs').where('timestamp', '>', start_date).where('timestamp', '<', end_d

root_agent > I'm sorry, but there were no productivity logs found for the past week.


In [ ]:
response = await runner.run_debug("can you provide a summary of my productivity logs for the past week?")